<a href="https://colab.research.google.com/github/Rishabh690/Banking-_chatbot-/blob/main/Chatbot_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# !pip install Transformers
!pip install tensorflow gpt-2-simple

  Preparing metadata (setup.py) ... done
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24557 sha256=e73d741df05c3a4c1afcdb13bb3b674403d67ffac61c72271259541bcc57b11d
  Stored in directory: /root/.cache/pip/wheels/df/6a/fe/10d3223f78d1ac3e4c83bb4c5e2d28dfb1789c2fb4cc7ea8d0
Successfully built gpt-2-simple


In [2]:
df = pd.read_csv("/content/BankFAQs.csv")

In [3]:
df.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [4]:
df.count()

Question    1764
Answer      1764
Class       1764
dtype: int64

In [5]:
df=df.drop(["Class"],axis=1)

# gpt2 144m parameter

In [6]:
import gpt_2_simple as gpt2
import pandas as pd
import tensorflow as tf
import os
# Step 1: Set up the environment and load the pre-trained language model
model_name = "124M"  # You can try other GPT-2 models like "355M" for a larger one

# Download the GPT-2 model (if not already downloaded)
if not os.path.exists(os.path.join("models", model_name)):
    gpt2.download_gpt2(model_name=model_name)

# Create a TensorFlow session
sess = gpt2.start_tf_sess()


def preprocess_dataframe(df):
    conversations = df.apply(lambda row: f"User: {row['Question']}\nChatbot: {row['Answer']}\n", axis=1)
    return "".join(conversations)

data = preprocess_dataframe(df)

# Save the processed data to a text file
data_file = '/content/dataset.txt'
with open(data_file, 'w') as f:
    f.write(data)


gpt2.finetune(sess,
              dataset=data_file,
              model_name=model_name,
              steps=800,  # You can adjust the number of fine-tuning steps
              restore_from='fresh',
              run_name='fine_tuned_model_144M',
              )



Fetching checkpoint: 1.05Mit [00:00, 5.86Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 1.07Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 5.53Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:33, 15.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 876Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.21Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.42Mit/s]


Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


dataset has 170712 tokens
Training...
[1 | 9.64] loss=2.78 avg=2.78
[2 | 11.74] loss=2.66 avg=2.72
[3 | 13.85] loss=2.62 avg=2.68
[4 | 15.97] loss=2.90 avg=2.74
[5 | 18.09] loss=2.48 avg=2.69
[6 | 20.20] loss=2.47 avg=2.65
[7 | 22.33] loss=2.54 avg=2.63
[8 | 24.46] loss=2.55 avg=2.62
[9 | 26.61] loss=2.67 avg=2.63
[10 | 28.78] loss=2.66 avg=2.63
[11 | 30.92] loss=2.57 avg=2.63
[12 | 33.06] loss=2.77 avg=2.64
[13 | 35.21] loss=2.60 avg=2.63
[14 | 37.36] loss=2.53 avg=2.63
[15 | 39.52] loss=2.54 avg=2.62
[16 | 41.69] loss=2.09 avg=2.59
[17 | 43.85] loss=2.17 avg=2.56
[18 | 46.02] loss=1.96 avg=2.52
[19 | 48.19] loss=1.98 avg=2.49
[20 | 50.36] loss=2.03 avg=2.47
[21 | 52.54] loss=1.89 avg=2.44
[22 | 54.73] loss=2.13 avg=2.42
[23 | 56.91] loss=2.06 avg=2.40
[24 | 59.10] loss=2.53 avg=2.41
[25 | 61.29] loss=2.22 avg=2.40
[26 | 63.50] loss=2.51 avg=2.41
[27 | 65.70] loss=2.46 avg=2.41
[28 | 67.91] loss=2.27 avg=2.40
[29 | 70.13] loss=2.18 avg=2.39
[30 | 72.35] loss=2.40 avg=2.39
[31 | 74.58]

In [7]:
type(df)


pandas.core.frame.DataFrame

In [ ]:
# Step 4: Implement the conversation loop

faq_df=df
def get_matching_faqs(user_input, faq_df):

    matching_faqs = faq_df[faq_df['Question'].str.contains(user_input, case=False)]
    return matching_faqs['Question'].values.tolist()
print("Bank FAQ Chatbot: Hi, how can I assist you today?")
while True:
    user_input = input("You: ")

    # Check if the user wants to end the conversation
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Bank FAQ Chatbot: Goodbye! Have a nice day.")
        break

    # Get relevant FAQs from the knowledge base
    relevant_faqs = get_matching_faqs(user_input, faq_df)

    # Generate a chatbot response using the GPT-2 model
    chatbot_response = gpt2.generate(sess, model_name=model_name, prefix=user_input, length=50, return_as_list=True)[0]

    if relevant_faqs:
        print("Bank FAQ Chatbot: Here are some related FAQs:")
        for faq in relevant_faqs:
            print(f"- {faq}")

    print(f"Bank FAQ Chatbot: {chatbot_response}")

Bank FAQ Chatbot: Hi, how can I assist you today?
You: hello
Bank FAQ Chatbot: hello: I have a retirement plan. I have HDFC Life Capital Plus Platinum Card. I am 65 and have a retirement plan. I use this card daily for checking my account balance. I am also extremely jealous of the amount of money I have in
You: How can I obtain an IVR Password
Bank FAQ Chatbot: Here are some related FAQs:
- How can I obtain an IVR Password 
Bank FAQ Chatbot: How can I obtain an IVR Password  for my Account from a Company
Chatbot: You may obtain the IVR password for your Account from PhoneBanking and will soon be able to apply it for a One Time Purchase.
User: What is the HDFC Bank Friday Savings


# Gpt 2 355M parameters

In [ ]:
import gpt_2_simple as gpt2
import pandas as pd
import tensorflow as tf
import os
# Step 1: Set up the environment and load the pre-trained language model
model_name = "355M"  # You can try other GPT-2 models like "355M" for a larger one

# Download the GPT-2 model (if not already downloaded)
if not os.path.exists(os.path.join("models", model_name)):
    gpt2.download_gpt2(model_name=model_name)

# Create a TensorFlow session
sess = gpt2.start_tf_sess()


def preprocess_dataframe(df):
    conversations = df.apply(lambda row: f"User: {row['Question']}\nChatbot: {row['Answer']}\n", axis=1)
    return "".join(conversations)

data = preprocess_dataframe(df)

# Save the processed data to a text file
data_file = '/content/dataset.txt'
with open(data_file, 'w') as f:
    f.write(data)


gpt2.finetune(sess,
              dataset=data_file,
              model_name=model_name,
              steps=800,  # You can adjust the number of fine-tuning steps
              restore_from='fresh',
              run_name='fine_tuned_model_355M',
              )



In [ ]:
# Step 4: Implement the conversation loop

faq_df=df
def get_matching_faqs(user_input, faq_df):

    matching_faqs = faq_df[faq_df['Question'].str.contains(user_input, case=False)]
    return matching_faqs['Question'].values.tolist()
print("Bank FAQ Chatbot: Hi, how can I assist you today?")
while True:
    user_input = input("You: ")

    # Check if the user wants to end the conversation
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Bank FAQ Chatbot: Goodbye! Have a nice day.")
        break

    # Get relevant FAQs from the knowledge base
    relevant_faqs = get_matching_faqs(user_input, faq_df)

    # Generate a chatbot response using the GPT-2 model
    chatbot_response = gpt2.generate(sess, model_name=model_name, prefix=user_input, length=50, return_as_list=True)[0]

    if relevant_faqs:
        print("Bank FAQ Chatbot: Here are some related FAQs:")
        for faq in relevant_faqs:
            print(f"- {faq}")

    print(f"Bank FAQ Chatbot: {chatbot_response}")